## Chapter 16
---
# Logistic Regression

Despire being called a regression, logistic regression is actually a widely used supervised classification technique. 
Allows us to predict the probability that an observation is of a certain class

## 16.1 Training a Binary Classifier

## 16.2 Training a Multiclass Classifier

## 16.3 Reducing Variance Through Regularization

## 16.4 Training a Classifier on Very Large Data

## 16.5 Handling Imbalanced Classes